# Scraping subreddits


## Part II - EDA
---

We'll import all our required libraries up here.

In [1]:
#import libraries
import pandas as pd, numpy as np, requests, time, nltk, datetime as dt

#NLP
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import gensim.downloader as api #allows us to get word2vec anf glove embeddins that we need
from gensim.models.word2vec import Word2Vec
from transformers import pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


#classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, confusion_matrix, plot_confusion_matrix, \
                             recall_score, precision_score)
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.dummy import DummyClassifier


# easier to see full text with a bigger maxwidth:
pd.options.display.max_colwidth = 200

In [2]:
#pip install python-Levenshtein
#this is for the gensim.similarities.levenshtein submodule

### Pull in the dataframes!

In [45]:
movie_deets2 = pd.read_csv('../data_outputs/good_movie_deets2.csv', index=False)
bad_deets2 = pd.read_csv('../data_outputs/bad_movie_deets2.csv', index=False)
all_movies = pd.read_csv('../data_outputs/all_movies.csv', index =False)

## Cleaning and EDA

Let's look at basic stats about our data.

In [32]:
#<df>.isnull().sum()

Let's understand the density and usefulness of our content, by analyzing the volume of comments and looking for the richness of the text-heavy columns we are expecting.

We are also going to see, whether there are any interesting prospective features for training our models.

In [33]:
#<df>['num_comments'].value_counts()

In [34]:
#check for post density
#<df>['selftext'].value_counts()

In [35]:
#<df>['title'].value_counts()

In [36]:
#<df>['author'].value_counts() #get only unique

## NLP & feature eng

### Word analysis

In [37]:
#word freq
#vectorize self-text

In [38]:
#sentiment